# Random Forest

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from fastai import *          # Quick access to most common functionality
from fastai.tabular import *  # Quick access to tabular functionality
from fastai.docs import *     # Access to example data provided with fastai

ModuleNotFoundError: No module named 'fastai.docs'

In [ ]:
??display

In [ ]:
PATH = "data/bulldozers/"

In [ ]:
!ls data/bulldozers

In [ ]:
!ls {PATH}